In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shashwatwork/knee-osteoarthritis-dataset-with-severity")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/knee-osteoarthritis-dataset-with-severity


In [2]:
import kagglehub
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image

In [3]:
path = "/kaggle/input/knee-osteoarthritis-dataset-with-severity"
train_dir = os.path.join(path, "train")
val_dir = os.path.join(path, "val")
test_dir = os.path.join(path, "test")

In [4]:
import torch.nn.functional as F

def asymmetric_penalty_loss(outputs, targets):
    # outputs: raw logits of shape (batch_size, num_classes)
    # targets: ground truth labels of shape (batch_size)

    log_probs = F.log_softmax(outputs, dim=1)  # Convert logits to log probabilities
    probs = torch.exp(log_probs)  # shape: (batch_size, num_classes)

    batch_size, num_classes = probs.shape
    range_tensor = torch.arange(num_classes, device=targets.device).unsqueeze(0).expand(batch_size, -1)

    # Expand targets to match output shape
    targets_expanded = targets.unsqueeze(1).expand_as(probs)

    # Compute the penalty matrix
    penalty = torch.ones_like(probs)
    penalty[range_tensor < targets_expanded] = 5  # Underestimation penalty
    # Overestimation or correct guess has penalty = 1.0

    # Get the loss: Negative Log Likelihood weighted by penalty
    loss = -penalty * log_probs
    loss = loss.gather(1, targets.unsqueeze(1)).squeeze(1)

    return loss.mean()


In [5]:
class KneeOADataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in range(5):  # Labels: 0 to 4
            label_dir = os.path.join(root_dir, str(label))
            if os.path.exists(label_dir):
                for img_name in os.listdir(label_dir):
                    self.image_paths.append(os.path.join(label_dir, img_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
train_dataset = KneeOADataset(train_dir, transform=transform)
val_dataset = KneeOADataset(val_dir, transform=transform)
test_dataset = KneeOADataset(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model2 = models.resnet18(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)
model2.fc = nn.Linear(num_ftrs, 5)
model2 = model2.to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Model with Cross Entropy Loss

In [11]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%")

# Save model
torch.save(model.state_dict(), "resnet_knee_oa.pth")

Epoch 1, Loss: 1.1662, Accuracy: 50.45%
Epoch 2, Loss: 0.9532, Accuracy: 60.35%
Epoch 3, Loss: 0.8902, Accuracy: 62.95%
Epoch 4, Loss: 0.8206, Accuracy: 65.54%
Epoch 5, Loss: 0.7779, Accuracy: 67.45%
Epoch 6, Loss: 0.7065, Accuracy: 70.58%
Epoch 7, Loss: 0.6234, Accuracy: 73.52%
Epoch 8, Loss: 0.5601, Accuracy: 76.70%
Epoch 9, Loss: 0.5054, Accuracy: 78.75%
Epoch 10, Loss: 0.4068, Accuracy: 83.13%


## Model with Underestimation Penalty

In [12]:
epochs = 10
for epoch in range(epochs):
    model2.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model2(images)
        loss = asymmetric_penalty_loss(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%")

# Save model
torch.save(model2.state_dict(), "resnet_knee_oa2.pth")

Epoch 1, Loss: 2.0159, Accuracy: 20.25%
Epoch 2, Loss: 2.0167, Accuracy: 20.13%
Epoch 3, Loss: 2.0133, Accuracy: 20.30%
Epoch 4, Loss: 2.0141, Accuracy: 20.68%
Epoch 5, Loss: 2.0169, Accuracy: 20.37%
Epoch 6, Loss: 2.0148, Accuracy: 20.37%
Epoch 7, Loss: 2.0165, Accuracy: 20.53%
Epoch 8, Loss: 2.0181, Accuracy: 19.85%
Epoch 9, Loss: 2.0160, Accuracy: 20.15%
Epoch 10, Loss: 2.0171, Accuracy: 20.39%


## Model Evaluation

In [13]:
def evaluate_model(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item() * inputs.size(0)

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [14]:
val_loss, val_acc = evaluate_model(model, val_loader, asymmetric_penalty_loss, device)
print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")


Validation Loss: 1.4103, Accuracy: 57.87%


In [15]:
test_loss, test_acc = evaluate_model(model, test_loader, asymmetric_penalty_loss, device)
print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_acc:.2f}%")


Test Loss: 1.1652, Accuracy: 62.32%


## Confusion Matrix

In [16]:
from sklearn.metrics import confusion_matrix
import numpy as np

def get_confusion_matrix(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds, labels=[0, 1, 2, 3, 4])
    return cm

In [17]:
get_confusion_matrix(model, test_loader, device)

array([[491,  74,  70,   4,   0],
       [144,  67,  75,  10,   0],
       [ 66,  69, 266,  46,   0],
       [  1,   7,  32, 165,  18],
       [  0,   0,   2,   6,  43]])

In [18]:
get_confusion_matrix(model2, test_loader, device)

array([[  0,   5, 369, 176,  89],
       [  0,   6, 171,  72,  47],
       [  0,   5, 272, 113,  57],
       [  0,   0, 133,  57,  33],
       [  0,   0,  27,  13,  11]])